## Notebook to evaluate different models:
- Ours Multipa
- Ctaguchi Model
- Allosaraus Model
- ZIPA Model (Pending Env Issues and IceFall, K2 not on mac?)

### Pending: Zipa Model

### Additional installation step for Epitran

```bash
$ git clone http://github.com/festvox/flite
$ cd flite
$ ./configure && make
$ sudo make install
$ cd testsuite
$ make lex_lookup
$ sudo cp lex_lookup /usr/local/bin
```

In [1]:
from pathlib import Path
import tempfile

import allosaurus.app
import allosaurus.bin.download_model
import datasets
import epitran
import numpy as np
import pandas as pd
import soundfile as sf
import transformers
import torch
from tqdm import tqdm

import multipa.data_utils
import multipa.evaluation


DEVICE = -1  # -1 for CPU, or set GPU index if available

# Paths For TIMIT Database and TIMIT IPA
# timit_data_dir = Path("/Users/parthbhangla/Desktop/Multipa_Datasets/TIMIT/COMPLETE")
# transcriptions_path = Path("/Users/parthbhangla/Desktop/Multipa_Datasets/TIMIT/complete_ipa.csv")
timit_data_dir = Path("../../data/TIMIT Dataset/COMPLETE")
transcriptions_path = Path("../../data/TIMIT Dataset/complete_ipa.csv")

# HuggingFace Models Evaluating
our_model = "ginic/full_dataset_train_3_wav2vec2-large-xlsr-53-buckeye-ipa"
taguchi_1k = "ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns"

# Set up results directories
RESULTS_DIR =Path("../../data/timit_results")
VERBOSE_RESULTS_DIR = RESULTS_DIR / "detailed_predictions"
AGGREGATE_METRICS_CSV = RESULTS_DIR / "aggregate_metrics/all_models_eval.csv"
EDIT_DIST_DIR = RESULTS_DIR / "edit_distances"
VERBOSE_RESULTS_DIR.mkdir(parents=True, exist_ok=True)
AGGREGATE_METRICS_CSV.parent.mkdir(parents=True, exist_ok=True)
EDIT_DIST_DIR.mkdir(parents=True, exist_ok=True)

# Processing options
IS_REMOVE_SPACES = True
NUM_PROC = 8 # Number of processes for HuggingFace dataset map and filter

# Computes and stores by-model performance metrics
model_evaluator = multipa.evaluation.ModelEvaluator()

evaluated_models = []

/Users/virginia/miniconda3/envs/multipa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_timit_gold_standard_transcriptions(transcriptions_path):
    """Returns a dictionary of {"audio_filename" -> {"ipa_transcription": transcription, "filename": original_filename}}"""
    gold_standard_df = pd.read_csv(transcriptions_path)
    gold_standard_df["filename"] = gold_standard_df["audio_filename"].str.lower()
    gold_standard_df.set_index("filename", inplace=True)
    return gold_standard_df.to_dict("index")


In [3]:
# Load TIMIT audio as a HuggingFace dataset with audio and gold standard transcriptions together
# This loads TIMIT as a Dataset with the same columns as the Buckeye corpus we've been working with
gold_standard_transcriptions = read_timit_gold_standard_transcriptions(transcriptions_path)

timit_wavs = [p for p in timit_data_dir.rglob("*") if p.suffix.lower() == ".wav"]
print("Total WAV files found:", len(timit_wavs))
data = []

for p in timit_wavs:
    clean_filename = "/" + str(p.relative_to(timit_data_dir.parent)).lower()
    ipa_transcription = gold_standard_transcriptions[clean_filename]["ipa_transcription"]

    entry = {
        "audio": {"path": str(p)},
        "filename": clean_filename,
        "ipa":ipa_transcription
    }
    data.append(entry)

audio_dataset = datasets.Dataset.from_list(data)
print(audio_dataset)
print(audio_dataset[0])

# TODO: Evaluate on the whole dataset
# Test with a small subset if wanted
audio_subset = audio_dataset.select([i for i in list(range(10))])
# audio_subset = audio_dataset

Total WAV files found: 6300
Dataset({
    features: ['audio', 'filename', 'ipa'],
    num_rows: 6300
})
{'audio': {'path': '../../data/TIMIT Dataset/COMPLETE/DR4/MMDM0/SI681.WAV'}, 'filename': '/complete/dr4/mmdm0/si681.wav', 'ipa': ' w ɨ d s ʌ tʃ ɨ n æ k t ɨ v ɹ ɨ f j ʉ ʒ l̩  b i j ʉ s f l̩  '}


In [4]:
# Sample audio correctly and preprocess transcriptions to remove whitepsace
audio_subset, audio_without_speech = multipa.evaluation.preprocess_test_data(audio_subset,
    is_remove_space=IS_REMOVE_SPACES, num_proc=NUM_PROC)
print("Audio with speech transcriptions")
print(audio_subset)
print(audio_subset[0])

# Sanity check that there's no audio without transcirptions
print("Audio without speech transcriptions")
print(audio_without_speech)


Filter (num_proc=8): 100%|██████████| 10/10 [00:00<00:00, 30.08 examples/s]


Audio with speech transcriptions
Dataset({
    features: ['audio', 'filename', 'ipa'],
    num_rows: 10
})
{'audio': {'path': '../../data/TIMIT Dataset/COMPLETE/DR4/MMDM0/SI681.WAV', 'array': array([-2.13623047e-04,  6.10351562e-05,  3.05175781e-05, ...,
       -3.05175781e-05, -9.15527344e-05, -6.10351562e-05]), 'sampling_rate': 16000}, 'filename': '/complete/dr4/mmdm0/si681.wav', 'ipa': 'wɨdsʌtʃɨnæktɨvɹɨfjʉʒl̩bijʉsfl̩'}
Audio without speech transcriptions
Dataset({
    features: ['audio', 'filename', 'ipa'],
    num_rows: 0
})


In [5]:
def allosaurus_predict(test_dataset, model="eng2102", phone_inventory="ipa"):
    print("Evaluating allosaurus. Model:", model, "Phone inventory:", phone_inventory)
    model_predictions = []
    recog = allosaurus.app.read_recognizer(model)
    for audio in tqdm(test_dataset["audio"]):
        wav_path = audio["path"]
        data, sr = sf.read(wav_path)
        with tempfile.NamedTemporaryFile(suffix=".wav") as tmp:
            sf.write(tmp.name, data, sr, format="WAV", subtype="PCM_16")
            prediction = recog.recognize(tmp.name, phone_inventory)
        #prediction = model.recognize(audio["path"], phone_inventory)
            model_predictions.append({multipa.evaluation.PREDICTION_KEY: prediction})
    predictions_dataset = datasets.Dataset.from_list(model_predictions)
    predictions_dataset = predictions_dataset.map(
        lambda x: multipa.data_utils.clean_text(x, text_key=multipa.evaluation.PREDICTION_KEY, is_remove_space=IS_REMOVE_SPACES), num_proc=NUM_PROC
    )
    return predictions_dataset

def hf_model_to_epitran_predict(model_name, test_dataset):
    print("Building pipeline and downloading model")
    if model_name.endswith(".en"):
        pipe = transformers.pipeline("automatic-speech-recognition", model=model_name, device=DEVICE)
    else:
        pipe = transformers.pipeline(
            "automatic-speech-recognition", model=model_name, generate_kwargs={"language": "english"}, device=DEVICE
        )
    print("Predicting with", model_name)
    orthography_predictions = [d["text"] for d in pipe(test_dataset["audio"])]
    epi = epitran.Epitran('eng-Latn')
    print("Transliterating with Epitran")
    ipa_predictions = []
    for pred in tqdm(orthography_predictions):
        result = epi.transliterate(pred)
        ipa_predictions.append({multipa.evaluation.PREDICTION_KEY: result})
    predictions_dataset = datasets.Dataset.from_list(ipa_predictions)
    predictions_dataset = predictions_dataset.map(
        lambda x: multipa.data_utils.clean_text(x, text_key=multipa.evaluation.PREDICTION_KEY, is_remove_space=IS_REMOVE_SPACES), num_proc=NUM_PROC
    )
    return predictions_dataset


In [6]:
# Allosaurus inference and metrics compute
allosaurus_model = "eng2102"
phone_inventory = "eng"
allosaurus_model_name = f"allosaurus_{allosaurus_model}_{phone_inventory}"
allosaurus.bin.download_model.download_model(allosaurus_model)
allosaurus_predictions = allosaurus_predict(audio_subset, model=allosaurus_model, phone_inventory=phone_inventory)
allosaurus_metrics = model_evaluator.eval_non_empty_transcriptions(allosaurus_model_name,
    allosaurus_predictions[multipa.evaluation.PREDICTION_KEY], audio_subset["ipa"])

# Write prediction details and edit distances
model_evaluator.write_edit_distance_results(allosaurus_model_name, EDIT_DIST_DIR)
multipa.evaluation.write_detailed_prediction_results(VERBOSE_RESULTS_DIR, allosaurus_model_name, audio_subset, allosaurus_predictions, allosaurus_metrics)

# Save model results for later
print("Done evaluating Allosaurus")
evaluated_models.append(allosaurus_model_name)
full_analysis_dataset = audio_subset.add_column(allosaurus_model_name, allosaurus_predictions[multipa.evaluation.PREDICTION_KEY])
print(full_analysis_dataset)

Evaluating allosaurus. Model: eng2102 Phone inventory: eng


/Users/virginia/miniconda3/envs/multipa/lib/python3.11/site-packages/allosaurus/am/utils.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(s

Done evaluating Allosaurus


Flattening the indices: 100%|██████████| 10/10 [00:00<00:00, 3357.86 examples/s]

Dataset({
    features: ['audio', 'filename', 'ipa', 'allosaurus_eng2102_eng'],
    num_rows: 10
})


In [7]:
# HuggingFace model inference and evaluation
models = [our_model, taguchi_1k]
for model_name in models:
    clean_model_name = multipa.evaluation.clean_model_name(model_name)
    print(f"Running ASR for model: {model_name}")
    asr_pipe = transformers.pipeline("automatic-speech-recognition", model=model_name, device=DEVICE)
    predictions_dataset = multipa.evaluation.get_clean_predictions(audio_subset, asr_pipe,
        num_proc=NUM_PROC, is_remove_space=IS_REMOVE_SPACES)

    # Compute all metrics
    model_metrics = model_evaluator.eval_non_empty_transcriptions(model_name,
        predictions_dataset[multipa.evaluation.PREDICTION_KEY], audio_subset["ipa"])

    # Write prediction details and edit distances
    model_evaluator.write_edit_distance_results(model_name, EDIT_DIST_DIR)
    multipa.evaluation.write_detailed_prediction_results(VERBOSE_RESULTS_DIR, clean_model_name, audio_subset, predictions_dataset, model_metrics)

    print("Done evaluating", model_name)
    evaluated_models.append(model_name)
    full_analysis_dataset = full_analysis_dataset.add_column(name=model_name, column=predictions_dataset[multipa.evaluation.PREDICTION_KEY])
    print(full_analysis_dataset)

Running ASR for model: ginic/full_dataset_train_3_wav2vec2-large-xlsr-53-buckeye-ipa


Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 625.74ba/s]


Done evaluating ginic/full_dataset_train_3_wav2vec2-large-xlsr-53-buckeye-ipa
Dataset({
    features: ['audio', 'filename', 'ipa', 'allosaurus_eng2102_eng', 'ginic/full_dataset_train_3_wav2vec2-large-xlsr-53-buckeye-ipa'],
    num_rows: 10
})
Running ASR for model: ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns


Some weights of the model checkpoint at ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.we

Done evaluating ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns
Dataset({
    features: ['audio', 'filename', 'ipa', 'allosaurus_eng2102_eng', 'ginic/full_dataset_train_3_wav2vec2-large-xlsr-53-buckeye-ipa', 'ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns'],
    num_rows: 10
})


In [ ]:
# Orthographic to epitran models
models = [
    "openai/whisper-large-v3-turbo",
    "openai/whisper-large-v3",
    "openai/whisper-medium.en",
]
for m in models:
    model_name = f"{m}_to_epitran".replace("/", "_")
    print("Evaulating", model_name)
    epitran_predictions = hf_model_to_epitran_predict(m, audio_subset)
    metrics = model_evaluator.eval_non_empty_transcriptions(
        model_name, epitran_predictions[multipa.evaluation.PREDICTION_KEY], audio_subset["ipa"]
    )
    multipa.evaluation.write_detailed_prediction_results(VERBOSE_RESULTS_DIR, model_name, audio_subset, epitran_predictions, metrics)
    model_evaluator.write_edit_distance_results(model_name, EDIT_DIST_DIR)
    print("Done evaluating", model_name)
    evaluated_models.append(model_name)
    full_analysis_dataset = full_analysis_dataset.add_column(name=model_name, column=epitran_predictions[multipa.evaluation.PREDICTION_KEY])
    print(full_analysis_dataset)

Evaulating openai_whisper-large-v3-turbo_to_epitran
Building pipeline and downloading model


/Users/virginia/miniconda3/envs/multipa/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=english, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=english.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Predicting with openai/whisper-large-v3-turbo
Transliterating with Epitran


100%|██████████| 10/10 [00:02<00:00,  4.97it/s]
Parameter 'function'=<function hf_model_to_epitran_predict.<locals>.<lambda> at 0x34a07e160> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 146.82ba/s]


Done evaluating openai_whisper-large-v3-turbo_to_epitran
Dataset({
    features: ['audio', 'filename', 'ipa', 'allosaurus_eng2102_eng', 'ginic/full_dataset_train_3_wav2vec2-large-xlsr-53-buckeye-ipa', 'ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns', 'openai_whisper-large-v3-turbo_to_epitran'],
    num_rows: 10
})
Evaulating openai_whisper-large-v3_to_epitran
Building pipeline and downloading model
Predicting with openai/whisper-large-v3


/Users/virginia/miniconda3/envs/multipa/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [ ]:
# Write all results to file for comparison
model_evaluator.to_csv(AGGREGATE_METRICS_CSV)

In [ ]:
print("These models were evaluated:", evaluated_models)
print("Dataset snippet for full anslysis:")
print(full_analysis_dataset)
print(full_analysis_dataset[0])

In [ ]:
predictions_df = full_analysis_dataset.to_pandas()
print(predictions_df.head())

# full_comparison_df = full_comparison_df.drop(
#     columns=["audio_filename", "ipa_transcription", "audio"]
# )

# full_comparison_df = full_comparison_df[[
#     "filename",
#     "ipa",
#     "ginic/data_seed_bs64_4_wav2vec2-large-xlsr-53-buckeye-ipa",
#     "ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns",
#     "allosaurus_eng2102_ipa"
# ]]

# print(full_comparison_df.head())

# output_path = Path("timit_subset_with_actual_and_predictions.csv")
# full_comparison_df.to_csv(output_path, index=False)
# print(f"Merged dataset saved to {output_path.resolve()}")

In [ ]:
gold_col = "ipa"
model_names = [our_model, taguchi_1k, "allosaurus_eng2102_ipa"]

def extract_dialect(path_str):
    path = Path(path_str)
    parts = [p for p in path.parts if p.lower().startswith("dr")]
    return parts[0].upper() if parts else "UNKNOWN"

full_comparison_df["dialect"] = full_comparison_df["filename"].apply(extract_dialect)
print("Dialect groups found:", full_comparison_df["dialect"].unique())

summary_data = {}
dialect_results = []

for model_name in model_names:
    print(f"Evaluating model: {model_name}")

    predictions = full_comparison_df[model_name].tolist()
    references = full_comparison_df[gold_col].tolist()

    metrics = model_eval.eval_non_empty_transcriptions(model_name, predictions, references)

    for metric_name in ["phone_error_rates", "phone_feature_error_rates", "feature_error_rates"]:
        col_name = f"{metric_name} VS {model_name}"
        full_comparison_df[col_name] = metrics[metric_name]

    summary_data[model_name] = {
        metric_name: float(np.mean(metrics[metric_name]))
        for metric_name in ["phone_error_rates", "phone_feature_error_rates", "feature_error_rates"]
    }

    for dialect, df_group in full_comparison_df.groupby("dialect"):
        result_row = {
            "dialect": dialect,
            "model": model_name,
        }
        for metric_name in ["phone_error_rates", "phone_feature_error_rates", "feature_error_rates"]:
            col_name = f"{metric_name} VS {model_name}"
            result_row[metric_name] = df_group[col_name].mean()
        dialect_results.append(result_row)


summary_df = pd.DataFrame(summary_data).T
summary_df = summary_df[["phone_error_rates", "phone_feature_error_rates", "feature_error_rates"]]
summary_df = summary_df.reset_index()
summary_df = summary_df.rename(columns={"index": "model"})
summary_df.to_csv("timit_model_evaluation_summary.csv", index=False)
print("Average evaluation metrics per model saved to timit_model_evaluation_summary.csv")


dialect_summary_df = pd.DataFrame(dialect_results)
dialect_summary_df.to_csv("timit_dialect_model_comparison.csv", index=False)
print("Dialect evaluation complete. Results saved to timit_dialect_model_comparison.csv")